## Importing the libraries:


In [1]:
import pandas as pd
from typing import Union, List, Dict
import numpy as np
import openpyxl
import os
import re
import xlsxwriter

## Importing the datasets:


In [2]:
# Importing dataset of balance and P&L function:
# Execution might take 10 minutes or more due to the large size of the datasets.

balance2025 = pd.read_excel('../../data/raw/firm-balance-statements/JAR_FA_RODIKLIAI_BLNS_2025.xlsx')
balance2024 = pd.read_excel('../../data/raw/firm-balance-statements/JAR_FA_RODIKLIAI_BLNS_2024.xlsx')
balance2023 = pd.read_excel('../../data/raw/firm-balance-statements/JAR_FA_RODIKLIAI_BLNS_2023.xlsx')
balance2022 = pd.read_excel('../../data/raw/firm-balance-statements/JAR_FA_RODIKLIAI_BLNS_2022.xlsx')
balance2021 = pd.read_excel('../../data/raw/firm-balance-statements/JAR_FA_RODIKLIAI_BLNS_2021.xlsx')
balance2020 = pd.read_excel('../../data/raw/firm-balance-statements/JAR_FA_RODIKLIAI_BLNS_2020.xlsx')

pnl2025 = pd.read_excel('../../data/raw/firm-PnL-statements/JAR_FA_RODIKLIAI_PLNA_2025.xlsx')
pnl2024 = pd.read_excel('../../data/raw/firm-PnL-statements/JAR_FA_RODIKLIAI_PLNA_2024.xlsx')
pnl2023 = pd.read_excel('../../data/raw/firm-PnL-statements/JAR_FA_RODIKLIAI_PLNA_2023.xlsx')
pnl2022 = pd.read_excel('../../data/raw/firm-PnL-statements/JAR_FA_RODIKLIAI_PLNA_2022.xlsx')
pnl2021 = pd.read_excel('../../data/raw/firm-PnL-statements/JAR_FA_RODIKLIAI_PLNA_2021.xlsx')
pnl2020 = pd.read_excel('../../data/raw/firm-PnL-statements/JAR_FA_RODIKLIAI_PLNA_2020.xlsx')

### Fallbacks for large dfs:

In [33]:

# Fallbacks for large dfs:
B2025 = balance2025.copy()
B2024 = balance2024.copy()
B2023 = balance2023.copy()
B2022 = balance2022.copy()
B2021 = balance2021.copy()
B2020 = balance2020.copy()

P2025 = pnl2025.copy()
P2024 = pnl2024.copy()
P2023 = pnl2023.copy()
P2022 = pnl2022.copy()
P2021 = pnl2021.copy()
P2020 = pnl2020.copy()

# Saving in lists for functions:
balance_list = [B2025, B2024, B2023, B2022, B2021, B2020]
pnl_list = [P2025, P2024, P2023, P2022, P2021, P2020]

## Cleaning the data with functions:
#### Removing unnecessary columns:

In [34]:
# Unnecessary column removal from list of dataframes:
def remove_mutual_unnecessary_columns(df_list):
    for df in df_list:
        remove_columns = ['ja_pavadinimas', 'obj_pav','form_pav','template_name',  'standard_name', 'form_pavadinimas','line_type_id', 'stat_pavadinimas', 'stat_pav']
        for col in remove_columns:
            if col in df.columns:
                df.drop(columns=col, inplace=True)
    return df_list

# Removing unnecessary rows from list of dataframes:
remove_mutual_unnecessary_columns(balance_list)
remove_mutual_unnecessary_columns(pnl_list)

# Renaming collumns to be the same across dataframes:
def rename_columns(df_list):
    for df in df_list:
        # Rename 'obj_kodas' to 'ja_kodas' if it exists
        if 'obj_kodas' in df.columns:
            df.rename(columns={'obj_kodas': 'ja_kodas'}, inplace=True)

        # Rename other columns if they exist
        column_mapping = {
            'nuosavas_kapitalas': 'NUOSAVAS KAPITALAS',
            'mok_sumos_ir_isipareigojimai': 'MOKĖTINOS SUMOS IR KITI ĮSIPAREIGOJIMAI',
            'trumpalaikis_turtas': 'TRUMPALAIKIS TURTAS',
            'ilgalaikis_turtas': 'ILGALAIKIS TURTAS',
            'pelnas_pries_apmokestinima': 'PELNAS (NUOSTOLIAI) PRIEŠ APMOKESTINIMĄ',
        'grynasis_pelnas': 'GRYNASIS PELNAS (NUOSTOLIAI)',
        'pardavimo_pajamos': 'PARDAVIMO PAJAMOS'
        }

        for old_col, new_col in column_mapping.items():
            if old_col in df.columns:
                df.rename(columns={old_col: new_col}, inplace=True)

    return df_list

rename_columns(balance_list)
rename_columns(pnl_list)




[         ja_kodas  form_kodas  stat_kodas template_id standard_id  \
 0       110003978         310           0      FS0329      IST024   
 1       110003978         310           0      FS0329      IST024   
 2       110003978         310           0      FS0329      IST024   
 3       110004884         310           0      FS0718      IST209   
 4       110004884         310           0      FS0718      IST209   
 ...           ...         ...         ...         ...         ...   
 423454  307123738         310           0      FS0329      IST024   
 423455  307193537         960           0      FS0522      IST024   
 423456  307193537         960           0      FS0522      IST024   
 423457  307438075         960           0      FS0522      IST118   
 423458  307438075         960           0      FS0522      IST118   
 
                                       line_name  reiksme beginning_date  \
 0                             PARDAVIMO PAJAMOS    97545     2024-01-01   
 1    

#### Extracting columns line_name, reiksme and ja_kodas:

In [35]:
# Extracting columns line_name, reiksme and ja_kodas from dfs with all of this data:
def extract_line_name_reiksme_ja_kodas(df_list):
    """
    Extract columns 'line_name', 'reiksme', and 'ja_kodas' from DataFrames
    that contain all three columns, and remove them from the original DataFrames.

    Parameters:
    -----------
    df_list : list of pandas.DataFrame
        List of DataFrames to process (will be modified in-place)

    Returns:
    --------
    list of pandas.DataFrame
        List of DataFrames containing only the three specified columns
    """
    extracted_dfs = []

    required_columns = ['line_name', 'reiksme', 'ja_kodas']

    for i, df in enumerate(df_list):
        # Check if all required columns exist in the current DataFrame
        if all(col in df.columns for col in required_columns):
            # Extract only the required columns
            extracted_df = df[required_columns].copy()
            extracted_dfs.append(extracted_df)

            # Remove the extracted columns from the original DataFrame
            df.drop(columns=required_columns, inplace=True)
            print(f"DataFrame {i}: Successfully extracted and removed columns")
        else:
            missing_cols = [col for col in required_columns if col not in df.columns]
            print(f"DataFrame {i}: Missing columns {missing_cols} - skipped")

    print(f"\nExtracted {len(extracted_dfs)} out of {len(df_list)} DataFrames")
    return extracted_dfs

B_extracted = extract_line_name_reiksme_ja_kodas(balance_list)
P_extracted = extract_line_name_reiksme_ja_kodas(pnl_list)

# Renaming the extracted dfs to be more descriptive:
B_extracted_2025 = B_extracted[0]
B_extracted_2024 = B_extracted[1]

P_extracted_2025 = P_extracted[0]
P_extracted_2024 = P_extracted[1]

# New lists with extracted and renamed dfs:
B_extracted_renamed = [B_extracted_2025, B_extracted_2024]
P_extracted_renamed = [P_extracted_2025, P_extracted_2024]

DataFrame 0: Successfully extracted and removed columns
DataFrame 1: Successfully extracted and removed columns
DataFrame 2: Missing columns ['line_name', 'reiksme'] - skipped
DataFrame 3: Missing columns ['line_name', 'reiksme'] - skipped
DataFrame 4: Missing columns ['line_name', 'reiksme'] - skipped
DataFrame 5: Missing columns ['line_name', 'reiksme'] - skipped

Extracted 2 out of 6 DataFrames
DataFrame 0: Successfully extracted and removed columns
DataFrame 1: Successfully extracted and removed columns
DataFrame 2: Missing columns ['line_name', 'reiksme'] - skipped
DataFrame 3: Missing columns ['line_name', 'reiksme'] - skipped
DataFrame 4: Missing columns ['line_name', 'reiksme'] - skipped
DataFrame 5: Missing columns ['line_name', 'reiksme'] - skipped

Extracted 2 out of 6 DataFrames


#### Shifting the column data of extraced dfs so that every row is unique with new columns:

In [43]:
# Shifting the column data of extraced dfs so that every row is unique:
def pivot_dfs(df_list):
    """
    Apply pivot transformation to multiple DataFrames.

    Parameters:
    -----------
    df_list : list of pandas.DataFrame
        List of DataFrames to pivot

    Returns:
    --------
    list of pandas.DataFrame
        List of pivoted DataFrames
    """

    def pivot_line_names_to_columns(df):
        """
        Pivot the DataFrame so that unique values in 'line_name' become separate columns.
        """
        # Pivot the table - use ja_kodas as index, line_name values as columns, and reiksme as values
        pivoted_df = df.pivot_table(
            index='ja_kodas',
            columns='line_name',
            values='reiksme',
            aggfunc='first'  # Use first() in case of duplicates
        ).reset_index()

        # Reset column names and clean up the DataFrame
        pivoted_df.columns.name = None  # Remove the 'line_name' label from columns

        return pivoted_df

    pivoted_dfs = []

    for i, df in enumerate(df_list):
        try:
            # Check if required columns exist
            required_columns = ['line_name', 'reiksme', 'ja_kodas']
            if not all(col in df.columns for col in required_columns):
                print(f"DataFrame {i}: Missing required columns. Available: {list(df.columns)}")
                pivoted_dfs.append(df)
                continue

            pivoted_df = pivot_line_names_to_columns(df)
            pivoted_dfs.append(pivoted_df)
            print(f"DataFrame {i}: Successfully pivoted. Original shape: {df.shape}, Pivoted shape: {pivoted_df.shape}")
        except Exception as e:
            print(f"DataFrame {i}: Error during pivoting - {e}")
            # Return original DataFrame if pivoting fails
            pivoted_dfs.append(df)

    return pivoted_dfs

pivot_dfs(B_extracted_renamed)
pivot_dfs(P_extracted_renamed)

# Renaming the pivoted dfs to be more descriptive:
B_pivoted_2025 = pivot_dfs(B_extracted_renamed)[0]
B_pivoted_2024 = pivot_dfs(B_extracted_renamed)[1]

P_pivoted_2025 = pivot_dfs(P_extracted_renamed)[0]
P_pivoted_2024 = pivot_dfs(P_extracted_renamed)[1]

# Saving in lists:
B_pivoted = pivot_dfs(B_extracted_renamed)
P_pivoted = pivot_dfs(P_extracted_renamed)


DataFrame 0: Successfully pivoted. Original shape: (615188, 3), Pivoted shape: (146512, 10)
DataFrame 1: Successfully pivoted. Original shape: (604271, 3), Pivoted shape: (141315, 10)
DataFrame 0: Successfully pivoted. Original shape: (423459, 3), Pivoted shape: (146512, 9)
DataFrame 1: Successfully pivoted. Original shape: (406755, 3), Pivoted shape: (141315, 9)
DataFrame 0: Successfully pivoted. Original shape: (615188, 3), Pivoted shape: (146512, 10)
DataFrame 1: Successfully pivoted. Original shape: (604271, 3), Pivoted shape: (141315, 10)
DataFrame 0: Successfully pivoted. Original shape: (615188, 3), Pivoted shape: (146512, 10)
DataFrame 1: Successfully pivoted. Original shape: (604271, 3), Pivoted shape: (141315, 10)
DataFrame 0: Successfully pivoted. Original shape: (423459, 3), Pivoted shape: (146512, 9)
DataFrame 1: Successfully pivoted. Original shape: (406755, 3), Pivoted shape: (141315, 9)
DataFrame 0: Successfully pivoted. Original shape: (423459, 3), Pivoted shape: (1465

#### Adding the pivoted dfs back to the original dfs:

In [45]:
# Adding the pivoted dfs back to the original dfs, wont write a function for this as it is a simple operation:

# Adding pivots back to balance data
B2025 = pd.merge(B2025, B_pivoted_2025, how='left')
print(f"B2025 shape: {B2025.shape}")

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

## Adding PnL data to balance data:

SyntaxError: closing parenthesis ']' does not match opening parenthesis '(' (3564248015.py, line 57)